In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob

In [2]:
DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

In [3]:
train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

In [4]:
from keras.preprocessing.text import Tokenizer

MAX_WORD_NUMS = 10000

word_tokenizer = Tokenizer(MAX_WORD_NUMS)
word_tokenizer.fit_on_texts(question_data["words"])

Using TensorFlow backend.


In [5]:
word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_WORD_NUMS]].values
    ),
    axis=0
)
word_embedding_data.shape

(10001, 300)

In [6]:
from keras.preprocessing.sequence import pad_sequences

WORD_SEQ_LEN = 30

def gen_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre")

In [7]:
word1, word2 = gen_data(train_data)
test_word1, test_word2 = gen_data(test_data)

In [8]:
# from keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization, Dense
# from keras.layers.merge import concatenate
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.callbacks import EarlyStopping, ModelCheckpoint

# from sklearn.model_selection import train_test_split

# train_word1, dev_word1, train_word2, dev_word2, train_y, dev_y = train_test_split(
#     word1, word2, train_data["label"].values,
#     test_size=0.2
# )

# train_word1.shape, dev_word1.shape

# lstm_size = 256
# drop_rate = 0.3
# dense_size = 300
# num_epoch = 100
# batch_size = 2048

# word_input1 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
# word_input2 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")

# embedding_layer = Embedding(
#     input_dim=word_embedding_data.shape[0],
#     output_dim=word_embedding_data.shape[1],
#     weights=[word_embedding_data],
#     input_length=WORD_SEQ_LEN,
#     trainable=False
# )

# word_vector1 = embedding_layer(word_input1)
# word_vector2 = embedding_layer(word_input2)

# lstm_layer = LSTM(lstm_size, dropout=drop_rate, recurrent_dropout=drop_rate)
# word1 = lstm_layer(word_vector1)
# word2 = lstm_layer(word_vector2)
# word = concatenate([word1, word2])

# x = Dropout(drop_rate)(word)
# x = BatchNormalization()(x)

# x = Dense(dense_size, activation="relu")(x)
# x = Dropout(drop_rate)(x)
# x = BatchNormalization()(x)

# pred = Dense(1, activation="sigmoid")(x)

# model = Model(inputs=[word_input1, word_input2], outputs=pred)
# model.compile(optimizer="nadam", loss="binary_crossentropy", metrics=["acc"])

# early_stop = EarlyStopping("val_loss", patience=10)
# check_point = ModelCheckpoint("./log/lstm.{epoch:02d}_{val_loss:.3f}.hdf5", monitor="val_loss", save_best_only=True, save_weights_only=True)

# train_res = model.fit(
#     x=[train_word1, train_word2],
#     y=train_y,
#     batch_size=batch_size,
#     epochs=num_epoch,
#     validation_data=([dev_word1, dev_word2], dev_y),
#     shuffle=True,
#     callbacks=[early_stop, check_point]
# )

# print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
# model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))

# test_pred = model.predict([test_word1, test_word2], batch_size=batch_size)
# pd.DataFrame(test_pred, columns=["y_pre"]).to_csv("./result/pred.csv", index=False)

In [9]:
from keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization, Dense
from keras.layers.merge import concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import StratifiedKFold

lstm_size = 256
drop_rate = 0.3
dense_size = 300
num_epoch = 100
batch_size = 2048

label = train_data["label"].values

preds = []

for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10).split(X=word1, y=label)):
    train_word1, train_word2, train_y = word1[train_index, :], word2[train_index, :], label[train_index]
    dev_word1, dev_word2, dev_y = word1[dev_index, :], word2[dev_index, :], label[dev_index]
    
    word_input1 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
    word_input2 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")

    embedding_layer = Embedding(
        input_dim=word_embedding_data.shape[0],
        output_dim=word_embedding_data.shape[1],
        weights=[word_embedding_data],
        input_length=WORD_SEQ_LEN,
        trainable=False
    )

    word_vector1 = embedding_layer(word_input1)
    word_vector2 = embedding_layer(word_input2)

    lstm_layer = LSTM(lstm_size, dropout=drop_rate, recurrent_dropout=drop_rate)
    word_output1 = lstm_layer(word_vector1)
    word_output2 = lstm_layer(word_vector2)
    word = concatenate([word_output1, word_output2])

    x = Dropout(drop_rate)(word)
    x = BatchNormalization()(x)

    x = Dense(dense_size, activation="relu")(x)
    x = Dropout(drop_rate)(x)
    x = BatchNormalization()(x)

    pred = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=[word_input1, word_input2], outputs=pred)
    model.compile(optimizer="nadam", loss="binary_crossentropy", metrics=["acc"])

    early_stop = EarlyStopping("val_loss", patience=10)
    check_point = ModelCheckpoint(
        "./log/lstm_%02d.{epoch:02d}_{val_loss:.3f}.hdf5" % (i + 1),
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True
    )

    train_res = model.fit(
        x=[train_word1, train_word2],
        y=train_y,
        batch_size=batch_size,
        epochs=num_epoch,
        validation_data=([dev_word1, dev_word2], dev_y),
        shuffle=True,
        callbacks=[early_stop, check_point]
    )
    
    print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
    model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))

    test_pred = model.predict([test_word1, test_word2], batch_size=batch_size)
    preds.append(pd.DataFrame(test_pred, columns=["y_pre"]))

pd.DataFrame(pd.concat(preds, axis=1).mean(axis=1), columns=["y_pre"]).to_csv("./result/pred.csv", index=False)

Train on 228946 samples, validate on 25440 samples
Epoch 1/100
228946/228946 [==============================] - 40s 176us/step - loss: 0.5117 - acc: 0.7532 - val_loss: 0.3931 - val_acc: 0.8292
Epoch 2/100
228946/228946 [==============================] - 39s 170us/step - loss: 0.3832 - acc: 0.8266 - val_loss: 0.3289 - val_acc: 0.8540
Epoch 3/100
228946/228946 [==============================] - 39s 170us/step - loss: 0.3381 - acc: 0.8499 - val_loss: 0.2928 - val_acc: 0.8690
Epoch 4/100
228946/228946 [==============================] - 39s 170us/step - loss: 0.3091 - acc: 0.8633 - val_loss: 0.2836 - val_acc: 0.8731
Epoch 5/100
228946/228946 [==============================] - 39s 170us/step - loss: 0.2897 - acc: 0.8737 - val_loss: 0.2678 - val_acc: 0.8801
Epoch 6/100
228946/228946 [==============================] - 39s 170us/step - loss: 0.2757 - acc: 0.8810 - val_loss: 0.2511 - val_acc: 0.8899
Epoch 7/100
228946/228946 [==============================] - 39s 171us/step - loss: 0.2632 - acc:

228946/228946 [==============================] - 40s 173us/step - loss: 0.1958 - acc: 0.9179 - val_loss: 0.2152 - val_acc: 0.9116
Epoch 34/100
228946/228946 [==============================] - 40s 173us/step - loss: 0.1966 - acc: 0.9175 - val_loss: 0.2214 - val_acc: 0.9105
Epoch 35/100
228946/228946 [==============================] - 40s 174us/step - loss: 0.1953 - acc: 0.9177 - val_loss: 0.2106 - val_acc: 0.9147
Epoch 36/100
228946/228946 [==============================] - 40s 174us/step - loss: 0.1949 - acc: 0.9178 - val_loss: 0.2116 - val_acc: 0.9148
Epoch 37/100
228946/228946 [==============================] - 40s 174us/step - loss: 0.1928 - acc: 0.9193 - val_loss: 0.2144 - val_acc: 0.9134
Epoch 38/100
228946/228946 [==============================] - 40s 174us/step - loss: 0.1925 - acc: 0.9195 - val_loss: 0.2124 - val_acc: 0.9153
Epoch 39/100
228946/228946 [==============================] - 40s 173us/step - loss: 0.1924 - acc: 0.9191 - val_loss: 0.2200 - val_acc: 0.9106
Epoch 40/100

load model ./log/lstm_03.34_0.219.hdf5
Train on 228947 samples, validate on 25439 samples
Epoch 1/100
228947/228947 [==============================] - 41s 178us/step - loss: 0.5138 - acc: 0.7531 - val_loss: 0.3969 - val_acc: 0.8274
Epoch 2/100
228947/228947 [==============================] - 39s 170us/step - loss: 0.3838 - acc: 0.8254 - val_loss: 0.3312 - val_acc: 0.8526
Epoch 3/100
228947/228947 [==============================] - 39s 172us/step - loss: 0.3387 - acc: 0.8493 - val_loss: 0.3027 - val_acc: 0.8665
Epoch 4/100
228947/228947 [==============================] - 40s 173us/step - loss: 0.3093 - acc: 0.8634 - val_loss: 0.2795 - val_acc: 0.8773
Epoch 5/100
228947/228947 [==============================] - 40s 173us/step - loss: 0.2904 - acc: 0.8738 - val_loss: 0.2650 - val_acc: 0.8851
Epoch 6/100
228947/228947 [==============================] - 40s 173us/step - loss: 0.2756 - acc: 0.8804 - val_loss: 0.2511 - val_acc: 0.8897
Epoch 7/100
228947/228947 [==============================]

228948/228948 [==============================] - 40s 175us/step - loss: 0.2090 - acc: 0.9114 - val_loss: 0.2276 - val_acc: 0.9057
Epoch 23/100
228948/228948 [==============================] - 40s 175us/step - loss: 0.2065 - acc: 0.9122 - val_loss: 0.2207 - val_acc: 0.9099
Epoch 24/100
228948/228948 [==============================] - 40s 175us/step - loss: 0.2056 - acc: 0.9138 - val_loss: 0.2291 - val_acc: 0.9061
Epoch 25/100
228948/228948 [==============================] - 40s 175us/step - loss: 0.2032 - acc: 0.9142 - val_loss: 0.2204 - val_acc: 0.9107
Epoch 26/100
228948/228948 [==============================] - 40s 175us/step - loss: 0.2033 - acc: 0.9141 - val_loss: 0.2278 - val_acc: 0.9078
Epoch 27/100
228948/228948 [==============================] - 40s 175us/step - loss: 0.2001 - acc: 0.9154 - val_loss: 0.2231 - val_acc: 0.9101
Epoch 28/100
228948/228948 [==============================] - 40s 176us/step - loss: 0.1988 - acc: 0.9159 - val_loss: 0.2226 - val_acc: 0.9097
Epoch 29/100

228948/228948 [==============================] - 39s 170us/step - loss: 0.3394 - acc: 0.8486 - val_loss: 0.3157 - val_acc: 0.8634
Epoch 4/100
228948/228948 [==============================] - 39s 172us/step - loss: 0.3114 - acc: 0.8628 - val_loss: 0.2932 - val_acc: 0.8736
Epoch 5/100
228948/228948 [==============================] - 40s 173us/step - loss: 0.2932 - acc: 0.8720 - val_loss: 0.2686 - val_acc: 0.8840
Epoch 6/100
228948/228948 [==============================] - 40s 173us/step - loss: 0.2796 - acc: 0.8784 - val_loss: 0.2662 - val_acc: 0.8867
Epoch 7/100
228948/228948 [==============================] - 40s 174us/step - loss: 0.2688 - acc: 0.8836 - val_loss: 0.2495 - val_acc: 0.8917
Epoch 8/100
228948/228948 [==============================] - 40s 174us/step - loss: 0.2600 - acc: 0.8881 - val_loss: 0.2441 - val_acc: 0.8962
Epoch 9/100
228948/228948 [==============================] - 40s 174us/step - loss: 0.2520 - acc: 0.8917 - val_loss: 0.2449 - val_acc: 0.8981
Epoch 10/100
22894

Epoch 20/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.2127 - acc: 0.9097 - val_loss: 0.2262 - val_acc: 0.9086
Epoch 21/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.2111 - acc: 0.9109 - val_loss: 0.2284 - val_acc: 0.9065
Epoch 22/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.2094 - acc: 0.9115 - val_loss: 0.2298 - val_acc: 0.9060
Epoch 23/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.2076 - acc: 0.9122 - val_loss: 0.2275 - val_acc: 0.9081
Epoch 24/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.2060 - acc: 0.9132 - val_loss: 0.2240 - val_acc: 0.9082
Epoch 25/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.2054 - acc: 0.9126 - val_loss: 0.2231 - val_acc: 0.9097
Epoch 26/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.2030 - acc: 0.9145 - val_loss: 0.2247 - val_acc: 0.9097

Epoch 32/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.1954 - acc: 0.9180 - val_loss: 0.2251 - val_acc: 0.9091
Epoch 33/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.1949 - acc: 0.9182 - val_loss: 0.2222 - val_acc: 0.9104
Epoch 34/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.1952 - acc: 0.9183 - val_loss: 0.2204 - val_acc: 0.9127
Epoch 35/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.1929 - acc: 0.9191 - val_loss: 0.2212 - val_acc: 0.9110
Epoch 36/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.1942 - acc: 0.9181 - val_loss: 0.2264 - val_acc: 0.9089
Epoch 37/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.1934 - acc: 0.9198 - val_loss: 0.2227 - val_acc: 0.9107
Epoch 38/100
228948/228948 [==============================] - 39s 171us/step - loss: 0.1920 - acc: 0.9190 - val_loss: 0.2220 - val_acc: 0.9112